In [2]:
import tensorflow as tf
from __future__ import print_function
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

#data prepare

In [3]:
#import data
digits = load_digits()
X= digits.data
y = digits.target
y = LabelBinarizer().fit_transform(y)
X_train , X_test, y_train,y_test = train_test_split(X,y,test_size=.3)


In [4]:
#define placegholder for input in NN
#这里的keep_prob是保留概率，即我们要保留的结果所占比例
#它作为一个placeholder，在run时传入， 
#当keep_prob=1的时候，相当于100%保留，也就是dropout没有起作用

with tf.name_scope('input'):
    keep_prob = tf.placeholder(tf.float32)
    xs = tf.placeholder(tf.float32,[None,64],name='x_input')
    ys = tf.placeholder(tf.float32,[None,10],name='y_input')

#define layer
def add_layer(inputs,in_size,out_size,layer_name,activation_function=None):
    Weights = tf.Variable(tf.random_uniform([in_size,out_size]))
    biases = tf.Variable(tf.zeros([1,out_size])+.1)
    Wx_plus_b = tf.matmul(Weights,inputs)+biases
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    tf.summary.histogram(layer_name+'/outputs',outputs)
    return outputs
#add output layer 
L1 = add_layer(xs,8*8,1,'L1',activation_function=tf.nn.tanh)#8x8
predicction = add_layer(L1,1,1,'L2',activation_function=tf.nn.softmax)

 #add output layer
#l1 = add_layer(xs, 64, 50, 'l1', activation_function=tf.nn.tanh)
#prediction = add_layer(l1, 50, 10, 'l2', activation_function=tf.nn.softmax)

#calculate loss between date & prediciton
cross_entropy  = tf.reduce_mean(-tf.reduce_sum(
                    ys*tf.log(predicction),reduction_indices=[1]))
#record it
tf.summary.scalar('loss',cross_entropy)
train_step = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(cross_entropy)

#get session to 初始化 graph
sess  = tf.Session()
init = tf.initialize_all_tables()

# 將視覺化輸出
merged = tf.summary.merge_all()

#summarize the train and tesst
train_writer = tf.summary.FileWriter("logs/train",sess.graph)
test_writer = tf.summary.FileWriter("log/test",sess.graph)

sess.run(init)
N=100
for i in range(N):
    train_result = sess.run(train_step,feed_dict={xs:X_train,ys:y_train,keep_prob:0.5})
    
    
    

In [11]:

from __future__ import print_function
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

# load data
digits = load_digits()
X = digits.data
y = digits.target
y = LabelBinarizer().fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)


def add_layer(inputs, in_size, out_size, layer_name, activation_function=None, ):
    # add one more layer and return the output of this layer
    Weights = tf.Variable(tf.random_normal([in_size, out_size]))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, )
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    # here to dropout
    Wx_plus_b = tf.nn.dropout(Wx_plus_b, keep_prob)
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b, )
    tf.summary.histogram(layer_name + '/outputs', outputs)
    return outputs


# define placeholder for inputs to network
keep_prob = tf.placeholder(tf.float32)
xs = tf.placeholder(tf.float32, [None, 64])  # 8x8
ys = tf.placeholder(tf.float32, [None, 10])

# add output layer
l1 = add_layer(xs, 64, 50, 'l1', activation_function=tf.nn.tanh)
prediction = add_layer(l1, 50, 10, 'l2', activation_function=tf.nn.softmax)

# the loss between prediction and real data
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
                                              reduction_indices=[1]))  # loss
tf.summary.scalar('loss', cross_entropy)
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

sess = tf.Session()
merged = tf.summary.merge_all()
# summary writer goes in here
train_writer = tf.summary.FileWriter("logs/train", sess.graph)
test_writer = tf.summary.FileWriter("logs/test", sess.graph)

# tf.initialize_all_variables() no long valid from
# 2017-03-02 if using tensorflow >= 0.12
if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
    init = tf.initialize_all_variables()
else:
    init = tf.global_variables_initializer()
sess.run(init)
for i in range(500):
    # here to determine the keeping probability
    sess.run(train_step, feed_dict={xs: X_train, ys: y_train, keep_prob: 0.5})
    if i % 50 == 0:
        # record loss
        train_result = sess.run(merged, feed_dict={xs: X_train, ys: y_train, keep_prob: 1})
        test_result = sess.run(merged, feed_dict={xs: X_test, ys: y_test, keep_prob: 1})
        train_writer.add_summary(train_result, i)

InvalidArgumentError: You must feed a value for placeholder tensor 'input/x_input' with dtype float
	 [[Node: input/x_input = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'input/x_input', defined at:
  File "C:\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Anaconda3\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "C:\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "C:\Anaconda3\lib\site-packages\tornado\ioloop.py", line 887, in start
    handler